In [3]:
#main

def main():
    drawingModule = mediapipe.solutions.drawing_utils   #modulo dibujo
    hands_module = mediapipe.solutions.hands            #modulo deteccion manos

    capture = cv2.VideoCapture(0)
    capture.set(3, 640)
    capture.set(4, 480)
    
    aiimg, start_time, timer_started, time_left_now, hold_for_play, draw_timer, game_over_text, computer_played = start_game()
    
    with hands_module.Hands(static_image_mode=False,            #instancia de la clase hands 
                            min_detection_confidence=0.7, min_tracking_confidence=0.4, max_num_hands=2) as hands:       #parameters de hands  
                                                       
        while True:

            imgBG = cv2.imread("Resources/BG3.png")             #imagen de fondo del juego
            imgQ = cv2.imread("Resources/quad.png", cv2.IMREAD_UNCHANGED)             #imagen de fondo del juego

            if timer_started:                                   #gestion timer inicio juego/espera por deteccion mano
                now_time = time.time()
                time_elapsed = now_time - start_time
                if time_elapsed >= 1:
                    time_left_now -= 1
                    start_time = now_time
                    if time_left_now <= 0:
                        hold_for_play = True
                        timer_started = False                   #acaba timer, atende jugada  
                                  
            ret, frame = capture.read()                                         #captura frame webcam con openCV
            results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))     #frame  --> pasado a detector hands
            move = "UNKNOWN"                                                    #jugada = "unknown" hasta que se detecta otro gesto

            if results.multi_hand_landmarks:                                    #control de l'objeto multi_hand_landmarks
                for hand_landmarks in results.multi_hand_landmarks:             
                    if hold_for_play or time.time() - draw_timer <= 2:          #por 2 segundos -  dibuja esqueleto mano
                        drawingModule.draw_landmarks(frame, hand_landmarks, hands_module.HAND_CONNECTIONS)
                    
                    fingers = ['THUMB', 'INDEX', 'MIDDLE', 'RING', 'PINKY']
                    current_state = ''
                    for finger in fingers:
                        if finger == 'THUMB':
                            thumb_status = get_thumb_status(hands_module, hand_landmarks)
                            current_state += "1" if thumb_status else "0"
                        else:
                            finger_status = get_finger_status(hands_module, hand_landmarks, finger)
                            current_state += "1" if finger_status else "0"
                    move = {"00000": "Piedra", "11111": "Papel", "01100": "Tijeras"}.get(current_state, "UNKNOWN")                               
                
                
                if hold_for_play and move != "UNKNOWN":
                    won, cmp_move, aiimg = calculate_game_state(move)
                    hold_for_play, draw_timer, game_over_text, computer_played = (False, time.time(),
                        f"Vicoria!" if won == 1 else f"Derrota!" if won == -1 else "Empate!",
                        f"   Com: {cmp_move} | Tu: {move}"  )
                    print(f"Player played {move}")
                    print(f"Resources/{aiimg}.png")

            font = cv2.FONT_HERSHEY_SIMPLEX         #FONT_HERSHEY_DUPLEX    

            if not hold_for_play and not timer_started:         # inzio gioco / game over
                cv2.putText(imgQ, game_over_text + " " + computer_played, (350, 630), font, 0.8, (0, 0, 0, 255),  2, cv2.LINE_4)                                

            label_text = "PULSE BARRA PARA JUGAR!"  # "PRESS SPACE TO START!"

            if hold_for_play:
                label_text = "HAZ TU JUGADA!" #"PLAY NOW!"                
            elif timer_started:
                label_text = "EMPIEZA EN... " + str(time_left_now) #"PLAY STARTS IN " + str(time_left_now)
                aiimg = 0           # img jugada com1
                
            cv2.putText(imgBG,  label_text, (540, 410), font, 0.45, (0, 0, 0), 2, cv2.LINE_4)       # "press space to play"

            imgScaled = cv2.resize(frame, (0, 0), None, 0.875, 0.875)
            imgScaled = imgScaled[:, 80:480]                            #cuadro de 420x420 pixel

            imgBG[234:654, 795:1195] = imgScaled
            imgAI = cv2.imread(f'Resources/{aiimg}.png', cv2.IMREAD_UNCHANGED)
            imgBG = cvzone.overlayPNG(imgBG, imgAI, (90, 230))   #old coord(149, 310)
            imgBG = cvzone.overlayPNG(imgBG, imgQ, (0, 0))  
            
            cv2.imshow("BG", imgBG)

            if cv2.waitKey(1) == 32:
                print("pressed space")
                start_time = time.time()
                timer_started = True
                time_left_now = 3

            if cv2.waitKey(1) == 27:
                #cv2.destroyAllWindows()
                #capture.release()
                break

    cv2.destroyAllWindows()
    capture.release()


if __name__ == "__main__":
    main()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [1]:
import cv2
import mediapipe
import time
import random
import cvzone

# funcion de detecion de las informiaciones sobre las manos

def get_finger_status(hands_module, hand_landmarks, finger_name):
    finger_id_map = {'INDEX': 8, 'MIDDLE': 12, 'RING': 16, 'PINKY': 20}

    finger_tip_y = hand_landmarks.landmark[finger_id_map[finger_name]].y
    finger_dip_y = hand_landmarks.landmark[finger_id_map[finger_name] - 1].y
    finger_mcp_y = hand_landmarks.landmark[finger_id_map[finger_name] - 2].y
    return finger_tip_y < finger_mcp_y
 
# detecta estado de pulgar

def get_thumb_status(hands_module, hand_landmarks):
    thumb_tip_x = hand_landmarks.landmark[hands_module.HandLandmark.THUMB_TIP].x
    thumb_mcp_x = hand_landmarks.landmark[hands_module.HandLandmark.THUMB_TIP - 2].x
    thumb_ip_x = hand_landmarks.landmark[hands_module.HandLandmark.THUMB_TIP - 1].x
    return thumb_tip_x > thumb_ip_x > thumb_mcp_x


# funcion que calcula el estado del juego, se produce una mossa random del computer 

def calculate_game_state(move):
    moves = ["Piedra", "Papel", "Tijeras"]
    wins = {"Piedra": "Tijeras", "Papel": "Piedra", "Tijeras": "Papel"}
    selected = random.randint(0, 2)
    print("Computer played " + moves[selected])

    if moves[selected] == move:
        return 0, moves[selected], selected+1

    if wins[move] == moves[selected]:
        return 1, moves[selected], selected+1

    return -1, moves[selected], selected+1


# funcion de set estado inicial de juego

def start_game():
    aiimg = 0
    start_time = 0.0
    timer_started = False
    time_left_now = 3
    hold_for_play = False
    draw_timer = 0.0
    game_over_text = ""
    computer_played = ""
    return (aiimg, start_time, timer_started, time_left_now, hold_for_play, draw_timer, game_over_text, computer_played )